## Tuning the hyperparameters of a neural network using EasyVVUQ

In this tutorial we will use the EasyVVUQ `GridSampler` to perform a grid search on the hyperparameters of a simple Keras neural network model, trained to recognize hand-written digits. This is the famous MNIST data set, of which 4 input features (of size 28 x 28) are show below. These are fed into a standard feed-forward neural network, which will predict the label 0-9.

**Note**: This tutorial always runs on the localhost. One possibility of performing the grid search on a remote supercomputer involves the use of FabSim, see the `hyperparameter_tuning_tutorial_with_fabsim.ipynb` notebook tutorial.

The (Keras) neural network script is located in `mnist/keras_mnist.template`, which will form the input template for the EasyVVUQ encoder. We will assume you are familiar with the basic EasyVVUQ building blocks. If not, you can look at the [basic tutorial](https://github.com/UCL-CCS/EasyVVUQ/blob/dev/tutorials/basic_tutorial.ipynb).

![](mnist/mnist_feats.png)

We need EasyVVUQ, TensorFlow and the TensorFlow data sets to execute this tutorial. If you need to install these, uncomment the corresponding line below.

In [12]:
# !pip install easyvvuq
# !pip install tensorflow
# !pip install tensorflow_datasets

In [13]:
import easyvvuq as uq
import os
import numpy as np
from easyvvuq.actions import CreateRunDirectory, Encode, Decode, ExecuteLocal, Actions

We now set some flags:

In [14]:
# Work directory, where the EasyVVUQ directory will be placed
WORK_DIR = '/tmp'
# target output filename generated by the code
TARGET_FILENAME = 'output.csv'
# EasyVVUQ campaign name
CAMPAIGN_NAME = 'grid_test'

As is standard in EasyVVUQ, we now define the parameter space. In this case these are 4 hyperparameters. There is one hidden layer with `n_neurons` neurons, a Dropout layer after the input and hidden layer, with dropout probability `dropout_prob_in` and `dropout_prob_hidden` respectively. We made the `learning_rate` tuneable as well.

In [15]:
params = {}
params["n_neurons"] = {"type":"integer", "default": 32}
params["dropout_prob_in"] = {"type":"float", "default": 0.0}
params["dropout_prob_hidden"] = {"type":"float", "default": 0.0}
params["learning_rate"] = {"type":"float", "default": 0.001}

These 4 hyperparameter appear as flags in the input template `mnist/keras_mnist.template`. Typically this is generated from an input file used by some simualtion code. In this case however, `mnist/keras_mnist.template` is directly our Python script, with the hyperparameters replaced by flags. For instance:

```python
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dropout($dropout_prob_in),
  tf.keras.layers.Dense($n_neurons, activation='relu'),
  tf.keras.layers.Dropout($dropout_prob_hidden),
  tf.keras.layers.Dense(10)
])
```

is simply the neural network construction part with flags for the dropout probabilities and the number of neurons in the hidden layer. The encoder reads the flags and replaces them with numeric values, and it subsequently writes the corresponding `target_filename=hyper_param_tune.py`:

In [16]:
encoder = uq.encoders.GenericEncoder('./mnist/keras_mnist.template', target_filename='hyper_param_tune.py')

What follows are standard steps in setting up an EasyVVUQ Campaign

In [17]:
# execute the runs locally
execute = ExecuteLocal('python3 hyper_param_tune.py')

# decode the output CSV files, with stored training and test accuracy values
output_columns = ["accuracy_train", "accuracy_test"]
decoder = uq.decoders.SimpleCSV(target_filename=TARGET_FILENAME, output_columns=output_columns)

# actions are 1) create run dirs, 2) encode input template, 3) execute runs, 4) decode output files
actions = Actions(CreateRunDirectory(root='/tmp', flatten=True), Encode(encoder), execute, Decode(decoder))

# create the EasyVVUQ main campaign object
campaign = uq.Campaign(
    name=CAMPAIGN_NAME,
    work_dir=WORK_DIR,
)

# add the param definitions and actions to the campaign
campaign.add_app(
    name=CAMPAIGN_NAME,
    params=params,
    actions=actions
)

As with the uncertainty-quantification (UQ) samplers, the `vary` is used to select which of the `params` we actually vary. Unlike the UQ samplers we do not specify an input probability distribution. This being a grid search, we simply specify a list of values for each hyperparameter. Parameters not in `vary`, but with a flag in the template, will be given the default value specified in `params`.

In [18]:
vary = {"n_neurons": [64, 128], "learning_rate": [0.005, 0.01, 0.015]}

**Note:** we are mixing integer and floats in the `vary` dict. Other data types (string, boolean) can also be used.

The `vary` dict is passed to the `Grid_Sampler`. As can be seen, it created a tensor product of all 1D points specified in `vary`. If a single tensor product is not useful (e.g. because it creates combinations of parameters that do not makes sense), you can also pass a list of different `vary` dicts. For even more flexibility you can also write the required parameter combinations to a CSV file, and pass it to the `CSV_Sampler` instead.

In [19]:
# create an instance of the Grid Sampler
sampler = uq.sampling.Grid_Sampler(vary)

# Associate the sampler with the campaign
campaign.set_sampler(sampler)

# print the points
print("There are %d points:" % (sampler.n_samples()))
sampler.points

There are 6 points:


[array([[64, 0.005],
        [64, 0.01],
        [64, 0.015],
        [128, 0.005],
        [128, 0.01],
        [128, 0.015]], dtype=object)]

Run the `actions` (create directories with `hyper_param_tune.py` files in it)

In [20]:
###############################
# execute the defined actions #
###############################

campaign.execute().collate()

2023-02-08 16:03:44.946331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 16:03:44.955703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 16:03:44.992926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

2023-02-08 16:03:51.501659: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 16:03:51.506190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 16:03:51.506230: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-08 16:03:51.506261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wouter-XPS-13-7390): /proc/driver/nvidia/version does not exist
2023-02-08 16:03:51.506664: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use t

Epoch 1/6
Epoch 1/6
Epoch 1/6


2023-02-08 16:03:52.631532: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 16:03:52.632048: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-08 16:03:52.632157: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wouter-XPS-13-7390): /proc/driver/nvidia/version does not exist
2023-02-08 16:03:52.632559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 16:03:52.949574: W tensorflow/compiler/xla/stream_exec

Epoch 1/6
Epoch 1/6
Epoch 1/6


...

2023-02-08 16:04:16.448009: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


469/469 [==============================] - 25s 19ms/step - loss: 0.4395 - sparse_categorical_accuracy: 0.8714 - val_loss: 0.2499 - val_sparse_categorical_accuracy: 0.9277
Epoch 2/6
469/469 [==============================] - 28s 17ms/step - loss: 0.3759 - sparse_categorical_accuracy: 0.8899 - val_loss: 0.2053 - val_sparse_categorical_accuracy: 0.9395
Epoch 2/6
408/469 [=========================>....] - ETA: 0s - loss: 0.5421 - sparse_categorical_accuracy: 0.8485

2023-02-08 16:04:20.977864: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:417] Shuffle buffer filled.


469/469 [==============================] - 6s 12ms/step - loss: 0.2182 - sparse_categorical_accuracy: 0.9376 - val_loss: 0.1825 - val_sparse_categorical_accuracy: 0.9449
Epoch 3/6
469/469 [==============================] - 31s 18ms/step - loss: 0.5106 - sparse_categorical_accuracy: 0.8572 - val_loss: 0.2834 - val_sparse_categorical_accuracy: 0.9222
Epoch 2/6
469/469 [==============================] - 5s 9ms/step - loss: 0.1650 - sparse_categorical_accuracy: 0.9526 - val_loss: 0.1451 - val_sparse_categorical_accuracy: 0.9584
Epoch 4/6
469/469 [==============================] - 5s 9ms/step - loss: 0.2626 - sparse_categorical_accuracy: 0.9258 - val_loss: 0.2281 - val_sparse_categorical_accuracy: 0.9353
Epoch 3/6
469/469 [==============================] - 5s 11ms/step - loss: 0.1325 - sparse_categorical_accuracy: 0.9621 - val_loss: 0.1244 - val_sparse_categorical_accuracy: 0.9632
Epoch 5/6
469/469 [==============================] - 5s 11ms/step - loss: 0.2146 - sparse_categorical_accuracy:

79/79 [==============================] - 0s 1ms/step - loss: 0.1031 - sparse_categorical_accuracy: 0.9693


In [21]:
data_frame = campaign.get_collation_result()
data_frame

,run_id,iteration,n_neurons,learning_rate,dropout_prob_in,dropout_prob_hidden,accuracy_train,accuracy_test
,0,0,0,0,0,0,0,0
0,1,0,64,0.005,0.0,0.0,0.958767,0.9562
1,2,0,64,0.010,0.0,0.0,0.974700,0.9679
2,3,0,64,0.015,0.0,0.0,0.976450,0.9693
3,4,0,128,0.005,0.0,0.0,0.963883,0.9609
4,5,0,128,0.010,0.0,0.0,0.976283,0.9689
5,6,0,128,0.015,0.0,0.0,0.982617,0.9712


Display the hyperparameters with the maximum test accuracy

In [22]:
print("Best hyperparameters with %.2f%% test accuracy:" % (data_frame['accuracy_test'].max().values * 100,))
data_frame.loc[data_frame['accuracy_test'].idxmax()][vary.keys()]

Best hyperparameters with 97.12% test accuracy:


,n_neurons,learning_rate
,0,0
5,128,0.015
